In [9]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

#美麗湯
from bs4 import BeautifulSoup as bs

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

#正規表達式
import re

# 下載檔案的工具
import wget

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options
)

# 建立儲存圖片、影片的資料夾
folderPath = 'project_gutenberg'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 放置爬取的資料
listData = []

In [13]:
# 前往古騰堡中文書籍頁面網址
def visit():
    driver.get('https://www.gutenberg.org/browse/languages/zh')

 # 取得中文書名與文字檔網址
def getZhNovels():
     # 等待元素載入完畢
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located( 
            (By.CSS_SELECTOR, 'div.pgdbbylanguage') 
        )
    )
    # 取得網頁原始碼
    html = driver.page_source
    #print(html)
    
    # 用美麗湯重新編譯
    soup = bs(html, 'lxml')  
    #print(soup)
    
    # 清空列表
    listData.clear()
    
    # 取得所有書名與文字檔網址
    div = soup.select("div.pgdbbylanguage > ul > li > a")
    
    # 設定正規表達式
    regex01 = r'[a-zA-Z]'
    
    
    for a in div:
        #print(a.get_text())
        #print(a['href'])
        
        #比對標題中是否有英文字母
        match = re.search(regex01,a.get_text())
        #print(match)

        #只新增標題沒有英文字母的中文書籍
        if(match == None):
            listData.append({
                'title': a.get_text(),
                'link': 'https://www.gutenberg.org'+ a['href']+'.txt.utf-8',
                'content': ''
            })
# 取得中文書內文
def getZhNovelText():
    
    regex02 = r'[\u4E00-\u9FFF《》，。：「」；、？！『』]+'

    for a in listData:
        #進入內文連結
        try:
            driver.get(a['link'])
            WebDriverWait(driver, 2).until(
            EC.presence_of_element_located( 
                (By.CSS_SELECTOR, 'body > pre') 
            )
            )
        except Exception:
            continue
        # 取得網頁原始碼
        html = driver.page_source
        
        # 用美麗湯重新編譯
        soup = bs(html, 'lxml')
        
        #取得內文
        div = soup.select_one("body > pre")
        txt = div.get_text()
        
        #只取中文內容
        match = re.findall(regex02,txt)
        
        #去掉頭三筆和最後一筆標題
        result = ''.join(match[3:-1])
        
        #存入檔案
        a['content'] = result
        

def close():
    driver.quit()

def writeZhNovelTextInFile():
    for a in listData:
        try:
            with open(f"{folderPath}/{a['title']}.txt", "w", encoding='utf-8') as file:
                file.write(a['content'])
        except Exception:
            print(a['title'])
            continue


In [14]:
visit()
getZhNovels()
getZhNovelText()
writeZhNovelTextInFile()
close()

臺灣通史
唐山過海的故事
